## Step 6: partition the files to follow the conventions KGTK uses for Wikidata

In [1]:
import sys  
sys.path.insert(0, 'tutorial')
from tutorial_setup import *
from generate_report import run

ALIAS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/aliases.en.tsv.gz"
ALL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/all.tsv.gz"
CLAIMS: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/claims.tsv.gz"
DESCRIPTION: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/descriptions.en.tsv.gz"
EXAMPLES_DIR: "/Users/amandeep/Github/kgtk/examples"
GE: "/Users/amandeep/Documents/kypher/temp.wikidata_os_v5/graph-embedding"
ISA: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.isa.tsv.gz"
ITEM: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/claims.wikibase-item.tsv.gz"
LABEL: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/labels.en.tsv.gz"
OUT: "/Users/amandeep/Documents/kypher/wikidata_os_v5"
P279: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.P279.tsv.gz"
P279STAR: "/Volumes/GoogleDrive/Shared drives/KGTK/datasets/wikidataos-v4/derived.P279star

In [2]:
%cd {output_path}

/Users/amandeep/Documents/kypher


We'll use the partition-wikidata notebook to complete this step. This notebook expects an input file that includes all edges and qualifiers together. We also need to specify a directory where partitioned files should be created, and a directory where temporary files can be sent (this should be different from our temp directory as the partition notebook will clear any existing files in this folder).

In [3]:
!mkdir -p $OUT/parts

In [4]:
!$kgtk cat -i $OUT/all.tsv.gz -i $OUT/qualifiers.tsv.gz -o $TEMP/all_and_qualifiers.tsv.gz

        5.83 real         5.64 user         0.11 sys


In [5]:
!zcat < $TEMP/all_and_qualifiers.tsv.gz | head

id	node1	label	node2
P10-P1628-32b85d-7927ece6-0	P10	P1628	"http://www.w3.org/2006/vcard/ns#Video"
P10-P1628-acf60d-b8950832-0	P10	P1628	"https://schema.org/video"
P10-P1629-Q34508-bcc39400-0	P10	P1629	Q34508
P10-P1659-P1651-c4068028-0	P10	P1659	P1651
P10-P1659-P18-5e4b9c4f-0	P10	P1659	P18
P10-P1659-P4238-d21d1ac0-0	P10	P1659	P4238
P10-P1659-P51-86aca4c5-0	P10	P1659	P51
P10-P1855-Q15075950-7eff6d65-0	P10	P1855	Q15075950
P10-P1855-Q69063653-c8cdb04c-0	P10	P1855	Q69063653
zcat: error writing to output: Broken pipe


In [6]:
pm.execute_notebook(
    os.environ["EXAMPLES_DIR"] + "/partition-wikidata.ipynb",
    os.environ["TEMP"] + "/partition-wikidata.out.ipynb",
    parameters=dict(
        wikidata_input_path = os.environ["TEMP"] + "/all_and_qualifiers.tsv.gz",
        wikidata_parts_path = os.environ["OUT"] + "/parts",
        temp_folder_path = os.environ["OUT"] + "/parts/temp",
        sort_extras = "--buffer-size 30% --temporary-directory $OUT/parts/temp",
        verbose = False
    )
)

Executing:   0%|          | 0/49 [00:00<?, ?cell/s]

{'cells': [{'cell_type': 'markdown',
   'metadata': {'tags': [],
    'papermill': {'exception': False,
     'start_time': '2021-01-14T18:06:01.856499',
     'end_time': '2021-01-14T18:06:01.878716',
     'duration': 0.022217,
     'status': 'completed'}},
   'source': '# Partitioning a subset of Wikidata\n\nThis notebook illustrates how to partition a Wikidata KGTK edges file.\n\nParameters are set up in the first cell so that we can run this notebook in batch mode. Example invocation command:\n\n```\npapermill partition-wikidata.ipynb partition-wikidata.out.ipynb \\\n-p wikidata_input_path /data3/rogers/kgtk/gd/kgtk_public_graphs/cache/wikidata-20201130/data/all.tsv.gz \\\n-p wikidata_parts_path /data3/rogers/kgtk/gd/kgtk_public_graphs/cache/wikidata-20201130/parts \\\n```\n\nHere is a sample of the records that might appear in the input KGTK file:\n```\nid\tnode1\tlabel\tnode2\trank\tnode2;wikidatatype\tlang\nQ1-P1036-418bc4-78f5a565-0\tQ1\tP1036\t"113"\tnormal\texternal-id\t\nQ1-P13

The partition-wikidata notebook created the following partitioned kgtk-files:

In [7]:
!ls $OUT/parts

aliases.en.tsv.gz                   metadata.property.datatypes.tsv.gz
aliases.tsv.gz                      metadata.types.tsv.gz
all.tsv.gz                          qualifiers.commonsMedia.tsv.gz
claims.commonsMedia.tsv.gz          qualifiers.external-id.tsv.gz
claims.external-id.tsv.gz           qualifiers.geo-shape.tsv.gz
claims.geo-shape.tsv.gz             qualifiers.globe-coordinate.tsv.gz
claims.globe-coordinate.tsv.gz      qualifiers.math.tsv.gz
claims.math.tsv.gz                  qualifiers.monolingualtext.tsv.gz
claims.monolingualtext.tsv.gz       qualifiers.musical-notation.tsv.gz
claims.musical-notation.tsv.gz      qualifiers.quantity.tsv.gz
claims.other.tsv.gz                 qualifiers.string.tsv.gz
claims.quantity.tsv.gz              qualifiers.tabular-data.tsv.gz
claims.string.tsv.gz                qualifiers.time.tsv.gz
claims.tabular-data.tsv.gz          qualifiers.tsv.gz
claims.time.tsv.gz                  qualifiers.url.tsv.gz
claims.tsv.gz                       quali

In [8]:
!$kypher -i $OUT/parts/claims.tsv.gz \
--match '(n1)-[]->()' \
--return 'count(distinct n1)'

count(DISTINCT graph_20_c1."node1")
16698


## Step 7 Run the Knowledge Graph Profiler

In [4]:
print(os.environ["USECASE_DIR"] + "/Knowledge-Graph-Profiler.ipynb")
print(os.environ["OUT"] + "/parts")
print(os.environ['TEMP'] + "/profiler_temp")
print(os.environ["OUT"] + "/profiler")

/Users/amandeep/Github/kgtk/use-cases/Knowledge-Graph-Profiler.ipynb
/Users/amandeep/Documents/kypher/wikidata_os_v5/parts
/Users/amandeep/Documents/kypher/temp.wikidata_os_v5/profiler_temp
/Users/amandeep/Documents/kypher/wikidata_os_v5/profiler


In [7]:
# the ; at the end suppresses the output of this cell which is a very large json object output of executing the profiler notebook
pm.execute_notebook(
    os.environ["USECASE_DIR"] + "/Knowledge-Graph-Profiler.ipynb",
    "Knowledge-Graph-Profiler.out.ipynb",
    parameters=dict(
        wikidata_parts_folder = os.environ["OUT"] + "/parts",
        cache_folder = os.environ['TEMP'] + "/profiler_temp",
        output_folder = os.environ["OUT"] + "/profiler",
        compute_graph_statistics = "true"
    )
)
;

Executing:   0%|          | 0/76 [00:00<?, ?cell/s]

''

[Knowledge Graph Profiler output](Knowledge-Graph-Profiler.out.ipynb)

### Generate a report on Profiler output

In [2]:
run(f'{os.environ["OUT"]}/profiler')

First_Growth example div creation failed
First_Growth incoming properties div creation failed
Alsace_wine example div creation failed
Alsace_wine incoming properties div creation failed


Click [here](report.html) to see the report

In [1]:
from IPython.core.display import display, HTML
display(HTML(open('report.html').read()))

Class_Label,Number of Instances
wine,"2,276"
First Growth,"1,078"
white wine,734
Alsace wine,722
beer brand,683
Other Classes,"8,305"
Property_Label,Number_of_Statements
Freebase ID,"1,677"
VIAF ID,750
Quora topic ID,710
